## Get Token

In [ ]:
import requests

url = "https://app.auger.ai/api/v1/"
email = '<email>'
password = '<password>'
params = {'email': email, 'password': password}
response = requests.post(url + "/tokens", json=params).json()
token = response.get('data').get('token')


In [ ]:
## Define Search Space

search_space = {
    "sklearn.ensemble.RandomForestClassifier": {
        "n_estimators": {
            "bounds": [50,300],
            "log": False,
            "type": "int",
            "tunable": True
        },
        "max_features": {
            "bounds": [0.05,1.0],
            "log": False,
            "type": "float",
            "tunable": True
        },
        "min_samples_split": {
            "bounds": [2,20],
            "log": False,
            "type": "int",
            "tunable": True
        },
        "min_samples_leaf": {
            "bounds": [1,20],
            "log": False,
            "type": "int",
            "tunable": True
        },
        "bootstrap": {
            "values": [True,False],
            "type": "categorical",
            "tunable": True
        }
    },
    "sklearn.ensemble.GradientBoostingClassifier": {
        "n_estimators": {
            "bounds": [50,300],
            "log": False,
            "type": "int",
            "tunable": True
        },
        "learning_rate": {
            "bounds": [0.001,1.0],
            "log": True,
            "type": "float",
            "tunable": True
        },
        "max_depth": {
            "bounds": [1,10],
            "log": False,
            "type": "int",
            "tunable": True
        },
        "min_samples_split": {
            "bounds": [2,20],
            "log": False,
            "type": "int",
            "tunable": True
        },
        "min_samples_leaf": {
            "bounds": [1,20],
            "log": False,
            "type": "int",
            "tunable": True
        },
        "subsample": {
            "bounds": [0.05,1.0],
            "log": False,
            "type": "float",
            "tunable": True
        },
        "max_features": {
            "bounds": [0.05,1.0],
            "log": False,
            "type": "float",
            "tunable": True
        }
    }
}

trial_count = 10



## Initialize Trial Search Space

In [ ]:

params = {'trials_total_count': trial_count, 'search_space': search_space, 'dataset_metafeatures':{}, 'token': token}
response = requests.post(url + "trial_searches", json=params).json()

data = response.get('data')

search_id = data.get('id')
print(search_id)



## Load Sample Dataset

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data.astype(np.float64),
    iris.target.astype(np.float64), train_size=0.75, test_size=0.25)



## Get Trials

In [ ]:
trials_limit = 4
def continue_trial_search(trials_history):
    data = {'trials_limit': trials_limit, 'trials_history': trials_history, 'token': token}
    response = requests.patch(url + "trial_searches/{}".format(search_id), json=data).json()

    return response.get('data').get('next_trials',[])



## Fit and Score Trials

In [ ]:
import copy
import importlib
import time


trials_history = []
next_trials = continue_trial_search(trials_history=trials_history)

while(len(trials_history) < trial_count and next_trials != None):
    for trial in next_trials:

        algo_params = copy.deepcopy(trial.get('algorithm_params'))

        module_name, class_name = trial.get('algorithm_name').rsplit('.', 1)
        cls = getattr(importlib.import_module(module_name), class_name)
        start_fit_time = time.time()
        model = cls(**algo_params).fit(X_train, y_train)


        history_item = copy.deepcopy(trial)
        history_item['evaluation_time'] = time.time() - start_fit_time

        history_item['score'] = model.score(X_test, y_test)

        trials_history.append(history_item)

    next_trials = continue_trial_search(trials_history=trials_history)
    
print(trials_history)